Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:

- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes (aka features)

Please answer the following questions:

1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")
2. Which mining techniques yielded the best results for what? How do you define "best results"?
3. How usefule was Principle Component Analysis? For what number of components did you get the "best results."

Please include tables and/or graphs to justify your statements about which are best.

Note: While numeric data is desired for classification/clustering, you need strings that look like market basket data
(sets of item names) for Association Rules. Numeric data can be grouped into names like "lowValue, midValue, midValue,
 mediumHighValue, highValue" etc. See the following code for possible way of doing this (not necessarily the best):

What to turn in:
- A pdf file with the answers to these three questions
- Your code used to answer these questions

Main Imports are below

In [77]:
# Importing Standard Libraries
import pandas as pd
import numpy as np
from csv import reader

# Sklearn Libraries
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.naive_bayes import *
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Importing mlxtend Libraries
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Graphing Libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
# Setting up view options
pd.set_option('display.max_columns', None)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.width = 0   #

In [79]:
# Importing the Data Set
df =  pd.read_csv("bank.csv")
# df =  pd.read_csv("bank-full.csv")
df.head()

# Establishing copies for running through the different scrubs
df_a = df.copy()
df_b = df.copy()

### Association Rules fore Data Mining
First Cleaning up the data and creating categories

In [80]:
for index, row in df_a.iterrows():
	if row['balance'] < 0:
		row['balance'] = 'inDebt'
	else:
		row['balance'] = 'notInDebt'

# Creating Categories for balance
df_a.loc[(df_a['balance'] >= 10000), 'balanceSummary'] = 'veryPositive'
df_a.loc[((df_a['balance'] < 0) & (df_a['balance'] >= -10000)), 'balanceSummary'] = 'negative'
df_a.loc[((df_a['balance'] < 10000) & (df_a['balance'] >= 0)), 'balanceSummary'] = 'positive'
df_a.loc[df_a['balance'] < -500, 'balanceSummary'] = 'veryNegative'

# Creating Categories for ages
df_a.loc[((df_a['age'] < 25) & (df_a['age'] >= 0)),  'ageBand'] = 'ageBand1'
df_a.loc[((df_a['age'] < 30) & (df_a['age'] >= 25)), 'ageBand'] = 'ageBand2'
df_a.loc[((df_a['age'] < 40) & (df_a['age'] >= 30)), 'ageBand'] = 'ageBand3'
df_a.loc[((df_a['age'] < 50) & (df_a['age'] >= 40)), 'ageBand'] = 'ageBand4'
df_a.loc[((df_a['age'] < 120) & (df_a['age'] >= 50)),'ageBand'] = 'ageBand5'

# Creating Categories for default
df_a.loc[(df_a['default'] == 'no'), 'defaultValue'] = 'defaultNo'
df_a.loc[(df_a['default'] == 'yes'), 'defaultValue'] = 'defaultYes'

# Creating Categories for Housing
df_a.loc[(df_a['housing'] == 'no'), 'housingVal'] = 'houseNo'
df_a.loc[(df_a['housing'] == 'yes'), 'housingVal'] = 'houseYes'

# Creating Categories for Loans
df_a.loc[(df_a['loan'] == 'no'), 'loanVal'] = 'loanNo'
df_a.loc[(df_a['loan'] == 'yes'), 'loanVal'] = 'loanYes'

# Displaying Head
df_a.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no,positive,ageBand3,defaultNo,houseYes,loanYes
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no,positive,ageBand3,defaultNo,houseYes,loanNo
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo


Dropping data that is not in a string and re-displaying dataframe head

In [81]:
df_a = df_a.drop(['age', 'day', 'duration', 'balance', 'campaign', 'previous', 'default', 'housing', 'loan', 'pdays'],
                 axis=1)
df_a.head()

,job,marital,education,contact,month,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,unemployed,married,primary,cellular,oct,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo
1,services,married,secondary,cellular,may,failure,no,positive,ageBand3,defaultNo,houseYes,loanYes
2,management,single,tertiary,cellular,apr,failure,no,positive,ageBand3,defaultNo,houseYes,loanNo
3,management,married,tertiary,unknown,jun,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
4,blue-collar,married,secondary,unknown,may,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo


Displaying all Unique Values

In [82]:
for col in df_a:
    print(col, ":", df_a[col].unique())

job : ['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
marital : ['married' 'single' 'divorced']
education : ['primary' 'secondary' 'tertiary' 'unknown']
contact : ['cellular' 'unknown' 'telephone']
month : ['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
poutcome : ['unknown' 'failure' 'other' 'success']
y : ['no' 'yes']
balanceSummary : ['positive' 'negative' 'veryPositive' 'veryNegative']
ageBand : ['ageBand3' 'ageBand5' 'ageBand4' 'ageBand1' 'ageBand2']
defaultValue : ['defaultNo' 'defaultYes']
housingVal : ['houseNo' 'houseYes']
loanVal : ['loanNo' 'loanYes']


Saving DF_A to file so that the apriori Algorithm will run faster

In [83]:
df_a.to_csv("outfile.csv", index=False)

Re-importing the df_a as file

In [84]:
file = []
with open("outfile.csv", 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        file.append(row)

Creating transact 1 hot boolean encoded numpy array and displaying data frame head

In [85]:
te = TransactionEncoder()
te_ary = te.fit(file).transform(file)
df_a_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_a_encoded.head()

,admin.,ageBand,ageBand1,ageBand2,ageBand3,ageBand4,ageBand5,apr,aug,balanceSummary,blue-collar,cellular,contact,dec,defaultNo,defaultValue,defaultYes,divorced,education,entrepreneur,failure,feb,houseNo,houseYes,housemaid,housingVal,jan,job,jul,jun,loanNo,loanVal,loanYes,management,mar,marital,married,may,month,negative,no,nov,oct,other,positive,poutcome,primary,retired,secondary,self-employed,sep,services,single,student,success,technician,telephone,tertiary,unemployed,unknown,veryNegative,veryPositive,y,yes
0,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False


Creating Filtering Function for frequent Items

In [86]:
def filter_freq_items(freq_items,items=1):
    """This function filters the frequent items so that we can look a certain n-itemsets"""
    filtered_freq_items = freq_items[freq_items["length"] == items].copy()
    return filtered_freq_items

Creating Frequent Items List and Association Rules

In [90]:
# Creating Frequent Items List
freq_items = apriori(df_a_encoded, min_support=.5, use_colnames=True)
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
freq_items.sort_values(["support", "length"], inplace=True, ascending=False)
freq_items.head()

,support,itemsets,length
1,0.982972,(defaultNo),1
6,0.898496,(positive),1
17,0.891199,"(positive, defaultNo)",2
5,0.884564,(no),1
16,0.869748,"(no, defaultNo)",2


In [91]:
# # # Creating Association Rules
rules = association_rules(freq_items, metric="confidence", min_threshold=0.3)
rules.sort_values(["lift"], inplace=True, ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
170,"(no, loanNo, defaultNo)","(unknown, positive)",0.731535,0.742149,0.561035,0.766929,1.033388,0.018127,1.106316
183,"(unknown, positive)","(no, loanNo, defaultNo)",0.742149,0.731535,0.561035,0.755959,1.033388,0.018127,1.100085
230,(no),(houseYes),0.884564,0.565900,0.517249,0.584750,1.033310,0.016674,1.045394
231,(houseYes),(no),0.565900,0.884564,0.517249,0.914029,1.033310,0.016674,1.342726
247,(no),"(houseYes, defaultNo)",0.884564,0.555949,0.507961,0.574250,1.032919,0.016189,1.042986
